<a href="https://colab.research.google.com/github/Gireesheruvaram/LLM-Beginer/blob/vector-database-kata2-branch/Kata2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import chromadb
import openai



In [ ]:
# Initialize ChromaDB client
client = chromadb.Client()

# Create a collection
collection = client.get_or_create_collection(
    name="hybrid_search",
    metadata={"description": "Hybrid search with metadata filtering"}
)


In [ ]:
#  Function to Generate Embeddings


def create_embedding(text):
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    return response['data'][0]['embedding']


In [ ]:
# Example documents with metadata
documents = [
    {
        "id": "doc1",
        "text": "This is a document about machine learning.",
        "metadata": {"author": "Alice", "date": "2023-01-01", "title": "Machine Learning Basics"}
    },
    {
        "id": "doc2",
        "text": "An in-depth guide to neural networks.",
        "metadata": {"author": "Bob", "date": "2023-02-15", "title": "Neural Networks"}
    }
]

# Add documents to the collection
for doc in documents:
    embedding = create_embedding(doc["text"])
    collection.add(
        ids=[doc["id"]],
        documents=[doc["text"]],
        metadatas=[doc["metadata"]],
        embeddings=[embedding]
    )


In [ ]:
def hybrid_search(query, metadata_filter=None, top_k=5):
    # Generate embedding for the query
    query_embedding = create_embedding(query)

    # Perform hybrid search with metadata filtering
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k,
        where=metadata_filter  # Metadata filter
    )

    return results


In [ ]:
# Define metadata filter (e.g., filter by author)
metadata_filter = {"author": "Alice"}

# Define query
query = "What is machine learning?"

# Perform search
results = hybrid_search(query, metadata_filter)

# Display results
for i, (doc, metadata) in enumerate(zip(results['documents'], results['metadatas'])):
    print(f"Result {i + 1}:")
    print(f"Document: {doc}")
    print(f"Metadata: {metadata}")
    print("-" * 50)


Result 1:
Document: ['This is a document about machine learning.']
Metadata: [{'author': 'Alice', 'date': '2023-01-01', 'title': 'Machine Learning Basics'}]
--------------------------------------------------
